In [17]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

# Define transformations
data_transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomAffine(15),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.GaussianBlur(3)

])

data_transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=data_transform_train
)

test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform=data_transform_test
)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64)

In [18]:
import torch.nn as nn
import torch.nn.functional as F
# Get cpu, gpu or mps device for training.

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [20]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, 1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(100352, 1024)
        self.bn4 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn5 = nn.BatchNorm1d(512)
        self.fc4 = nn.Linear(512, 102)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.bn1(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.bn2(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.bn3(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.bn4(x)
        x = F.relu(self.fc2(x))
        x = self.bn5(x)
        x = self.fc4(x)
        return self.logsoftmax(x)

model = NeuralNetwork().to(device)

learning_rate = 1e-5
batch_size = 64
epochs = 30

In [21]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
#scheduler= StepLR(optimizer, step_size=5, gamma=0.1)
scheduler = ReduceLROnPlateau(optimizer, mode='min',factor=0.1, patience=10, threshold=0.0001, threshold_mode='abs')

In [22]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # move data to device
        pred = model(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X) + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)  # move data to device
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

In [23]:
import time

loss_fn = nn.CrossEntropyLoss()

# Define early stopping parameters
patience = 10
min_delta = 0.001

best_loss = float('inf')
counter = 0

# Timer
start_time = time.time()

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    validation_loss = test_loop(test_dataloader, model, loss_fn)
    scheduler.step(validation_loss)

    # Early stopping logic
    if validation_loss < best_loss - min_delta:
        best_loss = validation_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break
            
end_time = (round(time.time()-start_time)/60, 3)

print("Done! This took", end_time, "minutes")


Epoch 1
-------------------------------
loss: 4.825345  [   64/ 1020]
Test Error: 
 Accuracy: 5.2%, Avg loss: 0.071318 

Epoch 2
-------------------------------
loss: 3.702777  [   64/ 1020]
Test Error: 
 Accuracy: 13.8%, Avg loss: 0.063759 

Epoch 3
-------------------------------
loss: 3.128741  [   64/ 1020]
Test Error: 
 Accuracy: 18.8%, Avg loss: 0.059621 

Epoch 4
-------------------------------
loss: 2.718049  [   64/ 1020]
Test Error: 
 Accuracy: 20.8%, Avg loss: 0.057838 

Epoch 5
-------------------------------
loss: 2.454848  [   64/ 1020]
Test Error: 
 Accuracy: 22.6%, Avg loss: 0.056686 

Epoch 6
-------------------------------
loss: 2.280874  [   64/ 1020]
Test Error: 
 Accuracy: 25.4%, Avg loss: 0.055483 

Epoch 7
-------------------------------
loss: 2.092327  [   64/ 1020]
Test Error: 
 Accuracy: 25.2%, Avg loss: 0.054681 

Epoch 8
-------------------------------
loss: 1.603945  [   64/ 1020]
Test Error: 
 Accuracy: 26.7%, Avg loss: 0.053919 

Epoch 9
-----------------

KeyboardInterrupt: 